# Mais sobre `SELECT`: agrupamentos

## Introdução

Vamos continuar nossa investigação sobre o comando `SELECT`. Desta vez usaremos a base de dados `'sakila'`, uma famosa base de dados de teste construída pelos autores do MySQL. 

Abra a URL [**https://dev.mysql.com/doc/sakila/en/**](https://dev.mysql.com/doc/sakila/en/) para conhecer melhor esta base de dados, que modela uma locadora de DVDs (riam, vocês também serão velhos um dia!).

Se você não possui esta base, siga as instruções de instalação na página ou faça o download direto pelo link [**https://downloads.mysql.com/docs/sakila-db.zip**](https://downloads.mysql.com/docs/sakila-db.zip)

Vamos também construir nosso objeto auxiliar para conectar com a base de dados, como fizemos na última aula.

In [10]:
from functools import partial
from dotenv import load_dotenv
import insperautograder.jupyter as ia
import mysql.connector
import os

load_dotenv(override=True)

connection = mysql.connector.connect(
    host=os.getenv("MD_DB_SERVER"),
    user=os.getenv("MD_DB_USERNAME"),
    password=os.getenv("MD_DB_PASSWORD"),
    port=int(os.getenv("MD_DB_PORT", 3306)),
    database="sakila"
)

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print("Executando query:")
        for result in cursor.execute(query, multi=True):
            if result.with_rows:
                for row in result.fetchall():
                    print(row)
            else:
                print(f"{result.rowcount} linhas afetadas.")

db = partial(run_db_query, connection)

## Exercícios para entrega

Esta aula tem atividade para entrega, confira os prazos e exercícios

In [ ]:
ia.tasks()

In [ ]:
ia.grades(task="group_having")

In [ ]:
ia.grades(by="task", task="group_having")

In [ ]:
# Nota em todas ATVs
ia.grades(by="task")

In [ ]:
# Média de ATV, dividindo por n-2
ia.average(excluded_count=2)

## Aquecimento

**Exercício 1**: Quais os nomes das categorias de filme? Ordene de forma crescente.

In [ ]:
sql_ex01 = """
-- SUA QUERY AQUI!
"""

db(sql_ex01)

In [ ]:
ia.sender(answer="sql_ex01", task="group_having", question="ex01", answer_type="pyvar")

**Exercício 2**: Quais atores tem as iniciais "J.D."? Exiba todos os atributos. Ordene de forma crescente pelo primeiro nome.

In [ ]:
sql_ex02 = """
-- SUA QUERY AQUI!
"""

db(sql_ex02)

In [ ]:
ia.sender(answer="sql_ex02", task="group_having", question="ex02", answer_type="pyvar")

**Exercício 3**: Liste as cidades brasileiras presentes na base de dados. Ordene de forma decrescente.

In [ ]:
sql_ex03 = """
-- SUA QUERY AQUI!
"""

db(sql_ex03)

In [ ]:
ia.sender(answer="sql_ex03", task="group_having", question="ex03", answer_type="pyvar")

**Exercício 4**: *Quantas* cidades brasileiras tem na base de dados? Renomeie para `qt_city`

In [ ]:
sql_ex04 = """
-- SUA QUERY AQUI!
"""

db(sql_ex04)

In [ ]:
ia.sender(answer="sql_ex04", task="group_having", question="ex04", answer_type="pyvar")

**Exercício 5**: Liste os filmes do ator (fictício) "Dan Harris". Ordene de forma crescente. Renomeie para `movie_title`.

In [ ]:
sql_ex05 = """
-- SUA QUERY AQUI!
"""

db(sql_ex05)

In [ ]:
ia.sender(answer="sql_ex05", task="group_having", question="ex05", answer_type="pyvar")

**Exercício 6**: Quais filmes estão alugados por Florence Woods?

Retorne:
- O primeiro nome
- O sobrenome
- O id do cliente
- O título do filme

Ordene pelo:
- Título do filme, decrescente

In [ ]:
sql_ex06 = """
-- SUA QUERY AQUI!
"""

db(sql_ex06)

In [ ]:
ia.sender(answer="sql_ex06", task="group_having", question="ex06", answer_type="pyvar")

**Exercício 7**: Para quais línguas não tem nenhum filme na locadora? Ordene de forma crescente.

**Dica**: use `LEFT OUTER JOIN`

In [ ]:
sql_ex07 = """
-- SUA QUERY AQUI!
"""

db(sql_ex07)

In [ ]:
ia.sender(answer="sql_ex07", task="group_having", question="ex07", answer_type="pyvar")

## `DISTINCT`

As vezes desejamos consultar quais os valores distintos de uma coluna. Para isso usamos o qualificador `DISTINCT`. 

Por exemplo: Quais os anos de lançamento dos filmes da base? 

In [ ]:
# Vai aparecer muitas cópias de "(2006,)"
db("SELECT release_year FROM film LIMIT 20")

Ops, parece que não tem muita variedade nesta base! Usando `DISTINCT` podemos limpar esse resultado:

In [ ]:
db("SELECT DISTINCT release_year FROM film")

### Praticando

**Exercício 8**: Quais clientes estão alugando um DVD agora? Ordene pelo nome e sobrenome. Retorne o id, nome e sobrenome do cliente.

In [ ]:
sql_ex08 = """
-- SUA QUERY AQUI!
"""

db(sql_ex08)

In [ ]:
ia.sender(answer="sql_ex08", task="group_having", question="ex08", answer_type="pyvar")

## Agrupamento

Uma das características mais valiosas de banco de dados é o *agrupamento*. Podemos agrupar os resultados de uma query indicando uma coluna cujos valores serão usados para agrupar os dados.

Por exemplo, considere a seguinte tabela, que chamaremos de `vendas`:

| id | id_item | item | preco |
|--|--|--|--|
| 1 | 1 | A | 5 |
| 2 | 2 | B | 6 |
| 3 | 1 | A | 3 |
| 4 | 3 | C | 7 |
| 5 | 3 | C | 5 |
| 6 | 1 | A | 2 |

Se agruparmos pela coluna `id_item` teremos 3 conjuntos de resultados:

id_item = 1:

| id | id_item | item | preco |
|--|--|--|--|
| 1 | 1 | A | 5 |
| 3 | 1 | A | 3 |
| 6 | 1 | A | 2 |

id_item = 2:

| id | id_item | item | preco |
|--|--|--|--|
| 2 | 2 | B | 6 |

id_item = 3:

| id | id_item | item | preco |
|--|--|--|--|
| 4 | 3 | C | 7 |
| 5 | 3 | C | 5 |

É como se tivessemos uma lista de tabelas! Isso não é permitido em SQL. Temos que **resumir** a informação de cada uma das tabelas a uma linha só, o que significa que, para cada coluna, devemos escolher uma dessas opções:
- Resumir a informação da coluna usando uma **função de grupo**. Podemos somar, tirar a média, contar itens, concatená-los em uma única string, entre outras;
- Para colunas que se relacionam 1 para 1 com a coluna de agrupamento (como a coluna de agrupamento em si, ou a coluna `item` neste exemplo), manter este valor. Isso acontece frequentemente quando fazemos `JOIN`.
- Não incluir a coluna, caso contrário.

Neste exemplo, podemos tomar a seguinte decisão para cada coluna:
- `id`: descartar
- `id_item`: manter valor
- `item`: manter valor
- `preco`: vamos calcular a soma dos valores, e renomear esta informação para `total`

Com isso, obtemos a seguinte tabela:

| id_item | item | total |
|--|--|--|
| 1 | A | 10 |
| 2 | B | 6 |
| 3 | C | 12 |

Por fim, se não queremos id_item, ficamos com a seguinte tabela:

| item | total |
|--|--|
| A | 10 |
| B | 6 |
| C | 12 |

Para obter essa tabela podemos usar o seguinte comando SQL:

```SQL
SELECT 
    item, SUM(preco) as total 
FROM 
    vendas
GROUP BY
    id_item
```

Consulte o capítulo 9 do seu livro texto para conhecer mais sobre agrupamentos.

### Praticando

**Exercício 9**: Quais os 10 atores que mais apareceram em filmes?

Retorne o id, nome, sobrenome e a quantidade de filmes que o ator atua (nomeie esta coluna como `film_count`).

Ordene de forma descrescente pela quantidade, crescente pelo primeiro nome e crescente pelo segundo nome.

In [ ]:
sql_ex09 = """
-- SUA QUERY AQUI!
"""

db(sql_ex09)

In [ ]:
ia.sender(answer="sql_ex09", task="group_having", question="ex09", answer_type="pyvar")

## Pipeline do comando `SELECT`

Uma versão mais completa do `SELECT` (mas não inteiramente completa - consulte o manual do MySQL) é vista abaixo:

```
SELECT [DISTINCT] <select_header> 
FROM <source_tables>
WHERE <filter_expression>
GROUP BY <grouping_expressions>
HAVING <filter_expression>
ORDER BY <ordering_expressions>
LIMIT <count> 
OFFSET <count>
```

Você já deve ter percebido que o comando `SELECT` tem uma sequência própria de avaliação. Por exemplo, para saber quais filmes custam mais que 3 dinheiros, podemos escrever:

In [ ]:
db("""
SELECT
    COUNT(f.rental_rate)
FROM
    film f
WHERE
    f.rental_rate > 3
""")

Observe que o 'apelido' f para a tabela 'film' é definido na cláusula `FROM`, mas usado em `SELECT` e também em `WHERE`.

A ordem de execução do comando `SELECT` é aproximadamente como segue:

1. `FROM <source_tables>`: indica as tabelas que serão usadas nesta query e, conceitualmente, combina estas tabelas através de *produto cartesiano* em uma grande tabela. (Note o termo "*conceitualmente*" que usei: em termos de implementação da query este produto cartesiano raramente é construído.)

2. `WHERE <filter_expression>`: filtra linhas.

3. `GROUP BY <grouping_expressions>`: agrupa conjuntos de linhas.

4. `SELECT <select_heading>`: escolha de colunas e de agregados.

5. `HAVING <filter_expression>`: outra filtragem, esta aplicada apenas **depois** da agregação. Pode usar resultados do processo de agregação. Obriga o uso de `GROUP BY`.

6. `DISTINCT`: Elimina linhas duplicadas.

7. `ORDER BY`: ordena as linhas do resultado.

8. `OFFSET <count>`: Pula linhas do resultado. Requer LIMIT.

9. `LIMIT <count>`: Mantém apenas um número máximo de linhas.

Esta sequencia também serve como dica de como projetar uma query! 
- Comece identificando as tabelas que você deseja usar
- Monte o filtro de linhas, incluindo critérios de `JOIN`
- Agrupe
- Selecione colunas e aplique funções de agregação, conforme necessário
- Filtre com `HAVING`, agora que temos agregação
- O resto é mais fácil, aplique conforme requerido

## `WHERE` versus `HAVING`

Conforme visto acima, temos a cláusula `HAVING` para fazer filtragens *APÓS* agregação.

**Para que serve isso?** Por exemplo, suponha que queremos saber quais categorias de filme possuem a letra `a` em seu nome **E** menos que sessenta filmes cadastrados na base.

**A seguinte query irá falhar! Continue a leitura do *handout***

In [ ]:
db("""
SELECT 
    ca.category_id,
    ca.name,
    COUNT(f.film_id) AS film_count
FROM
    category ca
    INNER JOIN film_category fc USING(category_id)
    INNER JOIN film f USING(film_id)
WHERE
    lower(ca.name) LIKE '%a%' AND
COUNT(f.film_id) < 60 -- FALHA AQUI!!!!!!!!!!!!!!!!!!!!
GROUP BY
    ca.category_id
ORDER BY
    film_count DESC
""")

A query falha porque a função de agregação `COUNT` necessita que as linhas já tenham sido selecionadas / filtradas. Então, ao tentar utilizar no `WHERE` uma função que depende das linhas filtradas pelo `WHERE`, criamos uma situação de conflito!

Para solucionar, iremos utilizar `HAVING`:

In [ ]:
db("""
SELECT 
    ca.category_id,
    ca.name,
    COUNT(f.film_id) AS film_count
FROM
    category ca
    INNER JOIN film_category fc USING(category_id)
    INNER JOIN film f USING(film_id)
WHERE
    lower(ca.name) LIKE '%a%'
GROUP BY
    ca.category_id
HAVING
    film_count < 60
ORDER BY
    film_count DESC
""")

## Praticando

**Exercício 10**: Liste a duração média dos filmes na categoria 'Drama'. Renomeie o atributo retornado para `duracao_media_drama`.

In [ ]:
sql_ex10 = """
-- SUA QUERY AQUI!
"""

db(sql_ex10)

In [ ]:
ia.sender(answer="sql_ex10", task="group_having", question="ex10", answer_type="pyvar")

**Exercício 11**: Liste o nome da categoria e a duração média dos filmes por categoria. Renomeie o atributo de média para `avg_len`. Ordene de forma decrescente por `avg_len`.

In [ ]:
sql_ex11 = """
-- SUA QUERY AQUI!
"""

db(sql_ex11)

In [ ]:
ia.sender(answer="sql_ex11", task="group_having", question="ex11", answer_type="pyvar")

**Exercício 12**: Liste o nome da categoria e a duração média dos filmes por categoria, apenas para categorias cuja duração média de filme excede 120 minutos. Renomeie o atributo de média para `avg_len`. Ordene de forma decrescente por `avg_len`.

In [ ]:
sql_ex12 = """
-- SUA QUERY AQUI!
"""

db(sql_ex12)

In [ ]:
ia.sender(answer="sql_ex12", task="group_having", question="ex12", answer_type="pyvar")

**Exercício 13**: Quais atores participaram de 35 a 40 filmes (intervalo fechado)?

Retorne:

- Nome
- Sobrenome
- Quantidade de filmes

Ordene por:
- Quantidade de filmes (Decrescente)
- Se houver empate na quantidade de filmes, ordene de forma crescente pelo nome e sobrenome.

In [ ]:
sql_ex13 = """
-- SUA QUERY AQUI!
"""

db(sql_ex13)

In [ ]:
ia.sender(answer="sql_ex13", task="group_having", question="ex13", answer_type="pyvar")

# Conclusão

Esta aula de hoje foi bastante densa! Dicas de estudo:

- Pratique no seu livro-texto, capítulo 9. Lembre-se que a base 'música' pode ser usada para praticar os comandos SQL vistos no livro.
- Tente criar queries que sirvam de exemplo para os conceitos do livro! A tarefa de criar exemplos é muito instrutiva!

**Leitura prévia**:
Para a próxima aula vamos continuar praticando, com os assuntos dos capítulos 10 e 11, prepare-se para a aula, ok?

Até a próxima!

In [ ]:
connection.close()

## Conferir Notas

Confira se as notas na atividade são as esperadas!

Primeiro na atividade atual!

In [ ]:
ia.grades(by="task", task="group_having")

In [ ]:
ia.grades(task="group_having")

In [ ]:
ia.grades(by="task")

In [ ]:
# Média de ATV, dividindo por n-2
ia.average(excluded_count=2)